<a href="https://colab.research.google.com/github/KhanShaheb34/Bangla-Paraphrase-Identification/blob/master/models/Fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install fasttext
! gdown --id 1_C260zgutx-BXmMUkDYfhJTqlDLCsHPI
! unzip FinalDataset.zip

In [1]:
import fasttext.util
fasttext.util.download_model('bn', if_exists='ignore')
ft = fasttext.load_model('cc.bn.300.bin')

In [2]:
import re
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [3]:
def hide_numbers(df, token=' [CC] '):
  df.sentence1 = [re.sub(r"[^ঀ-ৣ ]", '', s).strip() for s in df.sentence1]
  df.sentence2 = [re.sub(r"[^ঀ-ৣ ]", '', s).strip() for s in df.sentence2]
  return df

def load_data():
    train_df = hide_numbers(pd.read_csv('train.csv'))
    test_df = hide_numbers(pd.read_csv('test.csv'))
    val_df = hide_numbers(pd.read_csv('val.csv'))

    return train_df, test_df, val_df

def process_dataset(df):
  x, y = [], []

  for sent1, sent2, label in zip(df.sentence1, df.sentence2, df.label):
    x.append(np.concatenate((
        ft.get_sentence_vector(sent1),
        ft.get_sentence_vector(sent2)
    )))
    y.append(label)

  return x, y

train_df, test_df, val_df = load_data()

In [4]:
train_x, train_y = process_dataset(train_df)
test_x, test_y = process_dataset(test_df)
val_x, val_y = process_dataset(val_df)

In [7]:
model = SVC()
model.fit(train_x, train_y)

SVC()

In [8]:
def report(model, x, y):
  preds = model.predict(x)
  print(classification_report(y, preds))

In [10]:
print("Val Performance:")
report(model, val_x, val_y)
print("Test Performance:")
report(model, test_x, test_y)

Val Performance:
              precision    recall  f1-score   support

           0       0.66      0.91      0.77       872
           1       0.72      0.33      0.45       602

    accuracy                           0.67      1474
   macro avg       0.69      0.62      0.61      1474
weighted avg       0.69      0.67      0.64      1474

Test Performance:
              precision    recall  f1-score   support

           0       0.70      0.91      0.80       823
           1       0.78      0.44      0.57       571

    accuracy                           0.72      1394
   macro avg       0.74      0.68      0.68      1394
weighted avg       0.74      0.72      0.70      1394

